In [ ]:
import botorch
import gpytorch
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import torch
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

### Ideas

- Start with two dimensions and then scale from there (done)
- ARD kernel (done)
- Use batch BayesOpt (PoI done)
- Use a cost function so that energy consumption can not be less than 1
- Implement a non-fixed error term, i.e. allow error to vary (distributional regression)
- Compare models based on gpytorch metrics and mll

In [ ]:
building_energy = pd.read_excel("../../data/ENB2012_data.xlsx")

In [ ]:
feature_names = [
    "Relative Compactness", "Surface Area", "Wall Area", 
    "Roof Area", "Overall Height", "Orientation", 
    "Glazing Area", "Glazing Area Distribution", "Heating Load", 
    "Cooling Load"
]

building_energy.columns = feature_names
building_energy.columns = (building_energy.columns
                           .str.replace(' ', '_')
                           .str.lower()
                        )

building_energy["energy_consumption"] = building_energy[["heating_load", "cooling_load"]].sum(axis=1)

In [ ]:
X = building_energy.drop(["heating_load", "cooling_load", "energy_consumption"], axis=1)
y = building_energy["energy_consumption"]

In [ ]:
building_energy.describe().T

In [ ]:
# pairplot of the features
# sns.pairplot(building_energy, diag_kind="kde")

In [ ]:
building_energy.nunique()

## Gaussian process regression

With two features.

In [ ]:
X = building_energy[["relative_compactness", "surface_area"]]
y = building_energy["energy_consumption"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

min_max_scaler = MinMaxScaler()

X_train_scaled = torch.tensor(min_max_scaler.fit_transform(X_train), dtype=torch.float)
before = X_train_scaled.clone().detach()
X_test_scaled = torch.tensor(min_max_scaler.transform(X_test), dtype=torch.float)

y_train = torch.tensor(y_train.values, dtype=torch.float)
y_test = torch.tensor(y_test.values, dtype=torch.float)

In [ ]:
class GP(gpytorch.models.ExactGP, botorch.models.gpytorch.GPyTorchModel):
    num_outputs = 1
    def __init__(self, train_x, train_y, likelihood):
        super(GP, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ZeroMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(
                ard_num_dims=train_x.shape[1]
                )
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [ ]:
def fit_model(gp_model, num_iters=100):
    noise = 1e-4
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = gp_model(X_train_scaled, y_train, likelihood)
    model.likelihood.noise = noise
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    model.train()
    likelihood.train()

    loss_hist = []
    for _ in tqdm(range(num_iters)):
        optimizer.zero_grad()
        output = model(X_train_scaled)
        loss = -mll(output, y_train)
        loss_hist.append(loss.item())

        loss.backward()
        optimizer.step()
    
    return model, likelihood, loss_hist

In [ ]:
model, likelihood, loss_hist = fit_model(GP, num_iters=100)
plt.figure(figsize=(7, 3))
plt.plot(torch.arange(len(loss_hist)), loss_hist);

In [ ]:
grid = torch.linspace(0, 1, 100)
grid_x1, grid_x2 = torch.meshgrid(grid, grid, indexing='ij')
test_x = torch.stack([grid_x1.flatten(), grid_x2.flatten()], axis=1)
test_x

In [ ]:
model.eval()
likelihood.eval()

with torch.no_grad():
    predictive_dist = likelihood(model(test_x))
    predictive_mean = predictive_dist.mean
    predictive_std = predictive_dist.stddev

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))

c = ax[0].imshow(
    predictive_mean.detach().reshape(100, 100).transpose(-1, -2),
    origin="lower",
    extent=(0, 1, 0, 1),
)
ax[0].set_title("Predicted energy consumption $\\mu$")
plt.colorbar(c, ax=ax[0]);

c = ax[1].imshow(
    predictive_std.detach().reshape(100, 100).transpose(-1, -2),
    origin="lower",
    extent=(0, 1, 0, 1),
)
plt.colorbar(c, ax=ax[1])
ax[1].set_title("Predicted energy consumption $\\sigma$");

## BayesOpt

In [ ]:
X = building_energy[
    [
        "relative_compactness", "surface_area", "wall_area", 
        "roof_area", "overall_height", "orientation", 
        "glazing_area", "glazing_area_distribution"
     ]
    ]
y = building_energy["energy_consumption"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

min_max_scaler = MinMaxScaler()
std_scaler = StandardScaler()

X_train_scaled = torch.tensor(min_max_scaler.fit_transform(X_train), dtype=torch.float)
before = X_train_scaled.clone().detach()
X_test_scaled = torch.tensor(min_max_scaler.transform(X_test), dtype=torch.float)

y_train = torch.tensor(y_train.values, dtype=torch.float)
y_test = torch.tensor(y_test.values, dtype=torch.float)

In [ ]:
class RadialBasisGP(gpytorch.models.ExactGP, botorch.models.gpytorch.GPyTorchModel):
    num_outputs = 1
    def __init__(self, train_x, train_y, likelihood):
        super(RadialBasisGP, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(
                ard_num_dims=train_x.shape[1]
                )
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [ ]:
# metrics
# nlpd = gpytorch.metrics.negative_log_predictive_density(predictive_dist, y_test)
# mse = gpytorch.metrics.mean_squared_error(predictive_dist, y_test, squared=True)
# mae = gpytorch.metrics.mean_absolute_error(predictive_dist, y_test)

### Setting the domain bounds

Setting the bounds for the domain (search space) requires prior knowledge about the problem. It may not be desireable to set the lower and upper bound to the empirical min and max if it is possible to have values outside of these ranges. Doing so may restrict the domain, and thus the co-domain. Additionally, bounds may be set that are according to technical specifications. For example, belt speed, cutting angle, etc.

In [ ]:
# min_max bounds
bounds = torch.tensor(
    [
        [-2, -2, -2, -2, -2, -2, -2, -2],
        [2, 2, 2, 2, 2, 2, 2, 2],
    ],
    dtype=torch.float,
)

### Sequential Probability of Improvement

In [ ]:
# BayesOpt: minimize the objective function --> .min()
num_iters = 100
num_queries = 10
for i in range(num_queries):
    print("-"*20)
    print(f"iteration: {i}")
    print(f"incumbent: {X_train_scaled[y_train.argmin()]}, obj. func. value: {y_train.min():.4f}")

    model, likelihood, loss_hist = fit_model(RadialBasisGP, num_iters)

    policy = botorch.acquisition.analytic.ProbabilityOfImprovement(
            model, best_f=y_train.min()
        )

    next_x, acq_score = botorch.optim.optimize_acqf(
        policy,
        bounds=bounds,
        q=1,
        num_restarts=20*8,
        raw_samples=50*8
    )

    # evaluate the objective function and update training data
    with torch.no_grad():
        predictive_dist = likelihood(model(next_x))
        next_y_mean = predictive_dist.mean
        # predictive_upper, predictive_lower = predictive_dist.confidence_region()
    
    X_train_scaled = torch.cat([X_train_scaled, next_x])
    y_train = torch.cat([y_train, next_y_mean])

In [ ]:
plt.figure(figsize=(7, 3))
plt.plot(torch.arange(len(loss_hist)), loss_hist)
plt.xlabel("Iteration")
plt.ylabel("Marginal Log-Likelihood")
plt.title(f"Loss: {loss_hist[-1]:.4f}");

In [ ]:
minimizing_features = pd.DataFrame(X_train_scaled.numpy()[y_train.argmin()]).T
minimizing_features.columns = list(X_train.columns)
minimizing_features["objective_value"] = y_train.min().item()

In [ ]:
minimizing_features

### Batch Probability of Improvement

In [ ]:
# BayesOpt: minimize the objective function --> .min()
num_queries = 20
batch_size = 4
iters = num_queries // batch_size
for i in range(num_queries):
    print("-"*20)
    print(f"iteration: {i}")
    print(f"incumbent: {X_train_scaled[y_train.argmin()]}, obj. func. value: {y_train.min():.4f}")

    model, likelihood, loss_hist = fit_model(RadialBasisGP)

    policy = botorch.acquisition.monte_carlo.qProbabilityOfImprovement(
            model, best_f=y_train.min()
        )

    next_x, acq_score = botorch.optim.optimize_acqf(
        policy,
        bounds=bounds,
        q=batch_size,
        num_restarts=20*8,
        raw_samples=50*8
    )

    # evaluate the objective function and update training data
    with torch.no_grad():
        predictive_dist = likelihood(model(next_x))
        next_y_mean = predictive_dist.mean
        # predictive_upper, predictive_lower = predictive_dist.confidence_region()
    
    X_train_scaled = torch.cat([X_train_scaled, next_x])
    y_train = torch.cat([y_train, next_y_mean])

In [ ]:
plt.figure(figsize=(7, 3))
plt.plot(torch.arange(len(loss_hist)), loss_hist)
plt.xlabel("Iteration")
plt.ylabel("Marginal Log-Likelihood")
plt.title(f"Loss: {loss_hist[-1]:.4f}");

In [ ]:
minimizing_features = pd.DataFrame(X_train_scaled.numpy()[y_train.argmin()]).T
minimizing_features.columns = list(X_train.columns)
minimizing_features["objective_value"] = y_train.min().item()
minimizing_features

### Sequential constrained optimization

In [ ]:
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_model
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import ExpectedImprovement, ConstrainedExpectedImprovement
from botorch.optim import optimize_acqf
from botorch.utils import standardize

# Define the objective function (placeholder)
def objective_function(X):
    # Placeholder for the actual objective function
    return -torch.sum(X ** 2, dim=-1, keepdim=True)

# Generate some training data
train_X = torch.rand(10, 2, dtype=torch.double)
train_Y = objective_function(train_X)
train_Y = standardize(train_Y)  # standardize the output

# Define the Gaussian Process model
gp_model = SingleTaskGP(train_X, train_Y)

# Fit the GP model
mll = ExactMarginalLogLikelihood(gp_model.likelihood, gp_model)
fit_gpytorch_model(mll)

# Initialize the acquisition function with the outcome constraint
constraints = {0: (0.0, None)}
best_f = train_Y.max()
acq_function = ConstrainedExpectedImprovement(model=gp_model, best_f=best_f, objective_index=0, constraints=constraints)

# Optimize the acquisition function with constraints
bounds = torch.tensor([[0.0, 0.0], [1.0, 1.0]])
candidates, _ = optimize_acqf(
    acq_function=acq_function,
    bounds=bounds,
    q=1,
    num_restarts=5,
    raw_samples=20
)

print("Suggested candidate(s):")
print(candidates)


In [ ]:
bounds

## Dimensionality of grid predictions

Computational complexity $\mathcal{O}$ depends on the number of dimensions and the grandularity of the linspace.

In [ ]:
grid_x = torch.linspace(0, 1, 50)
grid_x1, grid_x2 = torch.meshgrid(grid_x, grid_x, indexing="ij")
xs = torch.vstack([grid_x1.flatten(), grid_x2.flatten()]).transpose(-1, -2)

In [ ]:
grid_x1.shape, grid_x2.shape, xs.shape

In [ ]:
50**2 # 2,500 rows and 2 dimensions (columns)

In [ ]:
grid_x = torch.linspace(0, 1, 10) # ONLY 10 points
grid_1, grid_2, grid_3, grid_4, grid_5, grid_6, grid_7, grid_8 = torch.meshgrid(
    [grid_x, grid_x, grid_x, grid_x, grid_x, grid_x, grid_x, grid_x], 
    indexing='ij'
)

In [ ]:
grid_1.shape, grid_1.flatten().shape

In [ ]:
10**8 # 100 million rows and 8 dimensions (columns)